In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn.datasets
import sklearn.model_selection
from sklearn.preprocessing import OneHotEncoder

# import mnist digits dataset
mnist = tf.keras.datasets.mnist # Object of the MNIST dataset
(x_train, y_train),(x_test, y_test) = mnist.load_data()

2023-04-30 17:31:43.988310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 17:31:44.104836: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-30 17:31:44.107711: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 17:31:45.373503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [28]:
# DATA TRUNCATION FOR TEST
x_train = x_train[:100]
y_train = y_train[:100]
x_test = x_test[:100]
y_test = y_test[:100]

In [29]:
# Normalize the train dataset
x_train = tf.keras.utils.normalize(x_train, axis=1)
# Normalize the test dataset
x_test = tf.keras.utils.normalize(x_test, axis=1)

# Hilbert Curve translation

In [30]:
# Hilbert path generator
# ALL CREDIT TO: https://github.com/jakubcerveny/gilbert/blob/master/gilbert2d.py
# ALL CREDIT TO @jakubcerveny on GitHub

def sgn(x):
    return -1 if x < 0 else (1 if x > 0 else 0)

def generate_path(x,y,ax,ay,bx,by):
    w = abs(ax + ay)
    h = abs(bx + by)

    (dax, day) = (sgn(ax), sgn(ay)) # unit major direction
    (dbx, dby) = (sgn(bx), sgn(by)) # unit orthogonal direction

    if h == 1:
        # trivial row fill
        for i in range(0, w):
            yield(x, y)
            (x, y) = (x + dax, y + day)
        return

    if w == 1:
        # trivial column fill
        for i in range(0, h):
            yield(x, y)
            (x, y) = (x + dbx, y + dby)
        return

    (ax2, ay2) = (ax//2, ay//2)
    (bx2, by2) = (bx//2, by//2)

    w2 = abs(ax2 + ay2)
    h2 = abs(bx2 + by2)

    if 2*w > 3*h:
        if (w2 % 2) and (w > 2):
            # prefer even steps
            (ax2, ay2) = (ax2 + dax, ay2 + day)

        # long case: split in two parts only
        yield from generate_path(x, y, ax2, ay2, bx, by)
        yield from generate_path(x+ax2, y+ay2, ax-ax2, ay-ay2, bx, by)

    else:
        if (h2 % 2) and (h > 2):
            # prefer even steps
            (bx2, by2) = (bx2 + dbx, by2 + dby)

        # standard case: one step up, one long horizontal, one step down
        yield from generate_path(x, y, bx2, by2, ax2, ay2)
        yield from generate_path(x+bx2, y+by2, ax, ay, bx-bx2, by-by2)
        yield from generate_path(x+(ax-dax)+(bx2-dbx), y+(ay-day)+(by2-dby),
                              -bx2, -by2, -(ax-ax2), -(ay-ay2))

def hilbert_path(n):
    yield from generate_path(0,0,n,0,0,n)


In [31]:
def dim_reduction(image):
    # Translate 2D image into 1D vector using Hilbert curve
    # image: 2D numpy array (28x28) or (nxn)
    # return: 1D numpy array
    width = image.shape[0]
    
    # Generate Hilbert curve path
    path = hilbert_path(width)
    # 1D vector widthxwidth
    vector = np.zeros(width*width)
    # Fill vector with image values
    for i, (x, y) in enumerate(path):
        vector[i] = image[x][y]
    return vector


In [32]:
# # Basic test
# print(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]]))
# print(dim_reduction(np.array([[0,0,.25,.25],[0,0,.25,.25],[.75,.75,1,1],[.75,.75,1,1]])))

### Translating MNIST

Note: This takes a long time!

In [33]:
x_train = np.array([dim_reduction(x) for x in x_train])
x_test = np.array([dim_reduction(x) for x in x_test])

### Converting to time series

In [34]:
train_width = 28*28*x_train.shape[0]
test_width = 28*28*x_test.shape[0]
x_train = x_train.flatten()
x_test = x_test.flatten()

In [35]:
y_train = np.array([[y for _ in range(28*28)] for y in y_train]).flatten()
y_test = np.array([[y for _ in range(28*28)] for y in y_test]).flatten()

In [36]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train.reshape(-1,1))

y_train = enc.transform(y_train.reshape(-1,1))
y_test = enc.transform(y_test.reshape(-1,1))

# Reservoir Computer